In [10]:
#Import libraries
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

%matplotlib inline

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
import pandas as pd

#Read input data
X_train = pd.read_csv('/content/drive/MyDrive/ML/titanic-ml-project/data/processed/train.csv')
X_test = pd.read_csv('/content/drive/MyDrive/ML/titanic-ml-project/data/processed/test.csv')

In [13]:
X_train

,Pclass,Sex,Age,Cabin,Fare_Per_Person,Family_Size_Encoded,Embarked_Encoded
0,3,1,-0.461538,0,-0.456311,6,0
1,1,0,0.769231,1,2.650485,6,2
2,3,0,-0.153846,0,-0.038835,4,0
3,1,0,0.538462,1,1.776699,6,0
4,3,1,0.538462,0,-0.029126,4,0
...,...,...,...,...,...,...,...
886,2,1,-0.076923,0,0.456311,4,0
887,1,0,-0.692308,1,2.106796,4,0
888,3,0,0.000000,0,-0.233010,8,0
889,1,1,-0.153846,1,2.106796,4,2


## Logistic Regression

In [14]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(random_state=16, max_iter=500)
lr.fit(X_train, Y_train)

NameError: name 'Y_train' is not defined

In [ ]:
Y_pred = lr.predict(X_test)
Y_pred

In [ ]:
diff_df = pd.DataFrame({'Actual' : Y_test, 'Predicted': Y_pred})
diff_df

In [ ]:
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(Y_test, Y_pred)
conf_matrix

 **Observations:**<br>
 **True Negatives (TN) = 121**<br>
The model correctly predicted 121 people as "No Diabetes."<br>
 **False Positives (FP) = 30**<br>
The model incorrectly predicted 30 people as having diabetes when they actually didn't (Type I error).<br>
 **False Negatives (FN) = 31**<br>
The model incorrectly predicted 31 people as "No Diabetes" when they actually had diabetes (Type II error).<br>
 **True Positives (TP) = 49**<br>
The model correctly predicted 49 people as having diabetes.<br>

In [ ]:
from sklearn.metrics import classification_report

target_names = ['without diabetes', 'with diabetes']
print(classification_report(Y_test, Y_pred, target_names=target_names))